# LAB | Connecting Python to SQL

In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:


### Challenge 

1. Establish a connection between Python and the Sakila database.

In [9]:
import mysql.connector
import getpass
import pandas as pd
from sqlalchemy import create_engine, text


In [4]:
def connect_to_database2(host, user, password, database):
    return create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

In [5]:
host="127.0.0.1"
user="root"
password=getpass.getpass()
database="sakila"

engine = connect_to_database2(host, user, password, database)


2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
- `month`: an integer representing the month for which rental data is to be retrieved.
- `year`: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [21]:
def rentals_month(engine, month, year):
    
    with engine.connect() as connection:    
        query = text(
            """SELECT 
                    *, 
                    DATE_FORMAT(rental_date, '%m') AS month, 
                    DATE_FORMAT(rental_date, '%Y') AS year 
                FROM sakila.rental
                WHERE DATE_FORMAT(rental_date, '%m') = :month
                AND DATE_FORMAT(rental_date, '%Y') = :year"""
        )
        result = connection.execute(query, {"month": str(month).zfill(2), "year": str(year)}) #zfill is zero-padded to ensure it matches the format used in the database, such as '05'
        df_rentals_month = pd.DataFrame(result.fetchall(), columns=result.keys())

    return df_rentals_month

In [24]:
df_rentals_month = rentals_month(engine, 5, 2005) # 5 and 2005 are integer
df_rentals_month.head(3)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,month,year
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,05,2005
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,05,2005
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,05,2005


3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [66]:
def rental_count_month(df_rentals_month, month, year):
    
    # month and year input are in integer, we need to convert it into str for naming the column:
    month_str = str(month).zfill(2) #the zfill(2) is to ensure that the month is in two digits
    year_str = str(year)

    rental_count = df_rentals_month.groupby('customer_id')['rental_id'].count().reset_index()

    rental_count.columns = ['customer_id', f'rentals_{month_str}_{year_str}']
     
    return rental_count

In [67]:
new_df = rental_count_month(df_rentals_month, 5, 2005)
new_df

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [82]:
def compare_rentals(df_1, df_2):#, month1, year1, month2, year2):
    

    df_merge = pd.merge(df_1, df_2, on='customer_id', how='outer').fillna(0)

    df_merge['difference'] = df_merge.iloc[:,1] - df_merge.iloc[:,2]

    return df_merge

In [85]:
df_merge = pd.merge(df_May_2005, df_June_2006, on='customer_id', how='outer').fillna(0)
df_merge.iloc[:,1]

0      2
1      1
2      2
3      3
4      3
      ..
515    4
516    1
517    6
518    2
519    1
Name: rentals_05_2005, Length: 520, dtype: int64

In [83]:
df_May_2005 = rental_count_month(df_rentals_month, 5, 2005)
df_June_2006 = rental_count_month(df_rentals_month, 6, 2006)
df_June_2006.head(3)

,customer_id,rentals_06_2006
0,1,2
1,2,1
2,3,2


In [84]:
df_with_difference = compare_rentals(df_May_2005, df_June_2006)#, 5, 2005, 6, 2006)
df_with_difference.head()

,customer_id,rentals_05_2005,rentals_06_2006,difference
0,1,2,2,0
1,2,1,1,0
2,3,2,2,0
3,5,3,3,0
4,6,3,3,0
